In [11]:
import os
import glob
import shutil
import sys
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi

In [13]:
# --- 1. Authenticate with Kaggle ---
api = KaggleApi()
api.authenticate()

# --- 2. Define directories ---
raw_root      = 'raw_csv'
incoming_dir  = os.path.join(raw_root, 'incoming')
processed_dir = os.path.join(raw_root, 'processed')
# Create directories if missing
os.makedirs(incoming_dir, exist_ok=True)
os.makedirs(processed_dir, exist_ok=True)

# --- 3. Attempt Kaggle download to incoming_dir ---
dataset = 'svaningelgem/crypto-currencies-daily-prices'
try:
    api.dataset_download_files(dataset, path=incoming_dir, unzip=True, force=False)
except Exception as e:
    print(f"Warning: Failed to download from Kaggle ({e}).\n" \
          f"Proceeding with files in '{incoming_dir}' and '{processed_dir}'.")

# --- 4. Collect raw CSV files ---
raw_files = glob.glob(os.path.join(incoming_dir, '*.csv'))
if not raw_files:
    # Fall back to processed_dir for initial or full reprocessing
    raw_files = glob.glob(os.path.join(processed_dir, '*.csv'))
    if raw_files:
        print(f"No new incoming files; using {len(raw_files)} files from processed_dir for full ingestion.")

if not raw_files:
    print(f"Error: No CSV files found in '{incoming_dir}' or '{processed_dir}'.\n" \
          "Please download or place raw CSVs accordingly.")
    sys.exit(1)

# --- 5. Concatenate all raw CSV files ---
df_list = [pd.read_csv(f) for f in raw_files]
df = pd.concat(df_list, ignore_index=True)

# --- 6. Clean and standardize data ---
# Strip whitespace from column names
df.columns = df.columns.str.strip()
# Map core columns to consistent names
target_map = {
    'date':   'Date',   'ticker': 'Ticker',
    'open':   'Open',   'high':   'High',
    'low':    'Low',    'close':  'Close'
}
col_map = {col: target_map.get(col.lower(), col) for col in df.columns}
df.rename(columns=col_map, inplace=True)
# Parse dates
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
# Drop duplicates
df.drop_duplicates(inplace=True)
# Drop any 'Unnamed' auto-index columns
unnamed = [c for c in df.columns if c.startswith('Unnamed')]
if unnamed:
    df.drop(columns=unnamed, inplace=True)

# --- 7. Archive new incoming files only ---
for f in glob.glob(os.path.join(incoming_dir, '*.csv')):
    shutil.move(f, os.path.join(processed_dir, os.path.basename(f)))

# --- 8. Save cleaned master file ---
master_file = 'master_crypto_daily_prices.csv'
sort_keys = [k for k in ['Ticker','Date'] if k in df.columns]
if len(sort_keys) == 2:
    df.sort_values(sort_keys, inplace=True)

df.to_csv(master_file, index=False)
print(f"Saved cleaned master with {len(df)} rows and {len(df.columns)} columns.")

Dataset URL: https://www.kaggle.com/datasets/svaningelgem/crypto-currencies-daily-prices
Proceeding with files in 'raw_csv/incoming' and 'raw_csv/processed'.
No new incoming files; using 106 files from processed_dir for full ingestion.
Saved cleaned master with 212719 rows and 6 columns.
